In [1]:
from pathlib import Path
import re
import warnings
import itertools

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
from scipy.sparse import csr_matrix
import scipy.stats as spss
import matplotlib_venn

import bioquest as bq
import sckit as sk

In [2]:
warnings.filterwarnings(action="ignore")
OUTPUT_DIR = "output/SC02.score"
Path(OUTPUT_DIR).mkdir(parents=True,exist_ok=True)
export = sk.export(formats=('pdf',),od=OUTPUT_DIR)

In [ ]:
adata = sc.read_h5ad("output/SC01.pp/adata.h5ad")

## SCORE

In [ ]:
CR = pd.read_csv("data/Circadian Clock pathcards.txt",index_col=0).index

In [ ]:
sk.aucell(adata,score_name="CR_Score",gene_list=CR)

In [38]:
sc.pl.umap(adata,color="CR_Score",show=False)
export("UMAP_CR_Score");

## DEG

In [ ]:
md = adata.obs["CR_Score"].median()
adata.obs["CR_Score_Group"] = adata.obs["CR_Score"].apply(lambda x: "High" if x > md else "Low")

In [39]:
sc.pl.umap(adata,color="CR_Score_Group",show=False,palette="Set2")
export("UMAP_CR_Group");

In [ ]:
adata.uns['log1p']={"base": None}
sk.deg(adata,groupby="CR_Score_Group",rank_name="CR_Score_Group")

In [ ]:
df=sk.deg_df(adata,rank_name="CR_Score_Group")

In [ ]:
High=bq.tl.subset(df,{"Identy":['High']})
High.to_csv(f"{OUTPUT_DIR}/CR_Score_Group_DEG_all.csv")
High=bq.tl.deg_siglabel(High,lfc='LogFC',padj="Padj",lfc_thr=(0.585, 0.585),pv_thr=(0.05, 0.05))
High.to_csv(f"{OUTPUT_DIR}/CR_Score_Group_DEG_sig.csv")

## enrich

In [ ]:
def enrich(gene_list,
           output_dir=None,
           fname=None,
           gene_sets=None,
           organism='human',
           pvalue_threshold=1.0,
           figsize=(6, 10),
           top_term=6,
           dotsize=5,
           ):
    import gseapy
    if gene_sets is None:
        gene_sets = {"GO_Biological_Process_2021": "GOBP",
                     "GO_Molecular_Function_2021": "GOMF",
                     "GO_Cellular_Component_2021": "GOCC",
                     "KEGG_{}".format("2019_Mouse" if organism == "mouse" else "2021_Human"): "KEGG"
                     }
    enr = gseapy.enrichr(
        gene_list=[x.upper() for x in gene_list],
        gene_sets=list(gene_sets.keys()),
        organism=organism,
        cutoff=pvalue_threshold
    )
    res = enr.results.replace(gene_sets)
    res.loc[:, "Term"] = bq.st.removes(string=res.Term, pattern=r"\(.+\)")
    ax = gseapy.dotplot(res,
                        column="Adjusted P-value",
                        x='Gene_set',  # set x axis, so you could do a multi-sample/library comparsion
                        size=dotsize,
                        top_term=top_term,
                        figsize=figsize,
                        title='',
                        xticklabels_rot=45,  # rotate xtick labels
                        show_ring=False,  # set to False to revmove outer ring
                        marker='o',
                        )
    ax.set_xlabel(xlabel="")
    if fname:
        import matplotlib.pyplot as plt
        plt.savefig(f"{output_dir}/{fname}_enrich.pdf",
                    bbox_inches='tight', dpi=300)
        res.to_csv(f"{output_dir}/{fname}_enrich.csv", index=False)


In [ ]:
deg_df = pd.read_csv(f"{OUTPUT_DIR}/CR_Score_Group_DEG_all.csv", index_col=0)
up = bq.tl.subset(deg_df, {"LogFC": "x>0.585", "Padj": "x<0.05"})
enrich(gene_list=up.index, output_dir=OUTPUT_DIR,
       fname="CR_Score_Group_UP", organism="human")

In [ ]:
deg_df = pd.read_csv(f"{OUTPUT_DIR}/CR_Score_Group_DEG_all.csv", index_col=0)
down = bq.tl.subset(deg_df, {"LogFC": "x<-0.585", "Padj": "x<0.05"})
enrich(gene_list=down.index, output_dir=OUTPUT_DIR,
       fname="CR_Score_Group_Down", organism="human")